In [3]:
import pandas as pd
import seaborn as sns

In [4]:
CPS_df = pd.read_csv("../data/processed/cps_2009_2017_clean")
ACS_df = pd.read_csv("../data/processed/ACS_2011_2017_clean")
NFCS_df = pd.read_csv("../data/processed/NFCS_2009_2018_clean")

C:\Users\Desmond\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
frames = [CPS_df, ACS_df, NFCS_df]

In [26]:
#declaring STATE list
STATES = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
          "Connecticut","Delaware","District of Columbia", "Florida","Georgia","Hawaii",
          "Idaho","Illinois", "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine",
          "Maryland","Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
          "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
          "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
          "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
          "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

In [27]:
#generating state:state_number dictionary
STATE_FIPS = list(frames[0].STATEFIP.unique())

STATE = {}
for state, name in zip(STATE_FIPS, STATES):
    STATE[state] = name

In [48]:
#generating STATE column for pertinent dfs
CPS_df["STATE"] = CPS_df.STATEFIP.map(STATE)
ACS_df["STATE"] = ACS_df.STATEFIP.map(STATE)

In [60]:
pop_prop = pd.read_csv("../data/interim/population_proportions")

In [72]:
pop_prop.BUNBANKED

0      9405
1      8083
2      8785
3      8929
4     56168
5     12088
6     11811
7      8890
8      8851
9     26018
10    14035
11    10085
12     8565
13    19681
14    10337
15    10546
16     9098
17     9354
18     9612
19     9548
20    13111
21    10969
22    15279
23    13168
24     8420
25    10466
26     8858
27     9387
28     9137
29    11679
30    13541
31     8075
32    27213
33    13757
34     8732
35    17667
36     8746
37     9741
38    19318
39     8907
40     9003
41     9119
42    10198
43    36518
44     9326
45     9109
46    13270
47    11663
48    10129
49    11673
50     8836
Name: BUNBANKED, dtype: int64

In [76]:
CPS_agg = pd.DataFrame()
CPS_agg["STATE"] = CPS_df.STATE
CPS_agg["UNDERBANKED"] = CPS_df.BUNBANKED
CPS_agg["UNDERBANKED_REASON"] = CPS_df.BUNDERBANKREAS

CPS_agg = CPS_agg.groupby(["STATE"]).count()

CPS_agg = CPS_agg.reset_index()

CPS_agg["STATE_SURVEY_POP"] = pop_prop["BUNBANKED"]

CPS_agg

,STATE,UNDERBANKED,UNDERBANKED_REASON,STATE_SURVEY_POP
0,Alabama,4783,1010,9405
1,Alaska,4398,1377,8083
2,Arizona,4909,1284,8785
3,Arkansas,4694,1031,8929
4,California,28842,8344,56168
5,Colorado,5991,2179,12088
6,Connecticut,6181,2350,11811
7,Delaware,4902,1625,8890
8,District of Columbia,5086,1305,8851
9,Florida,13291,3703,26018


In [97]:
CPS_agg = pd.DataFrame()
CPS_agg["STATE"] = CPS_df.STATE
CPS_agg["UNDERBANKED_REASON"] = CPS_df.BUNDERBANKREAS

CPS_agg = CPS_agg.groupby(["STATE"])["UNDERBANKED_REASON"].count()

CPS_agg = CPS_agg.reset_index()
CPS_agg
#CPS_agg[CPS_agg["STATE"] == "Alabama"][]

,STATE,UNDERBANKED_REASON
0,Alabama,1010
1,Alaska,1377
2,Arizona,1284
3,Arkansas,1031
4,California,8344
5,Colorado,2179
6,Connecticut,2350
7,Delaware,1625
8,District of Columbia,1305
9,Florida,3703


In [80]:
CPS_agg.reset_index()

,index,UNDERBANKED_REASON,STATE,STATE_SURVEY_POP
0,0,1.0,1060,9405
1,1,2.0,2607,8083
2,2,4.0,722,8785
3,3,6.0,166,8929
4,4,7.0,125,56168
5,5,8.0,355,12088
6,6,9.0,45,11811
7,7,10.0,861,8890
8,8,11.0,178,8851
9,9,96.0,46,26018


In [38]:
CPS_agg["STATE_SURVEY_POP"] = CPS_df["BUNBANKED"]

In [ ]:
CPS_agg = CPS_agg.reset_index()

In [39]:
CPS_agg.head()

,UNBANKED,STATE_SURVEY_POP
STATE,,
Alabama,4783,NaN
Alaska,4398,NaN
Arizona,4909,NaN
Arkansas,4694,NaN
California,28842,NaN
